# Trabalho Prático 1 - Geometria computacional

Autoras:

Deborah Santos Andrade Guimarães - Matrícula: 2014113810

Luísa Vasconcelos de Castro Toledo - Matrícula: 2020006795

Para aplicar os conhecimentos adquiridos em sala de aula sobre geometria computacional, foi proposto um trabalho em que se usasse os algoritmos de envoltória convexa e de verificação de interseçao de segmentos para o problema de classificação em aprendizado de máquina supervisionado. A seguir, está o relatório com detalhes sobre a escolha e a implementação dos algoritmos usados e os resultados do experimento realizado com a base de dados Iris.

## Envoltória convexa

O algoritmo de envoltória convexa utilizado foi o Gift Wrapping. Tem complexidade O(hn), em que n é o número de pontos no problema e h é o número de pontos na envoltória. Foi escolhido porque, dentre os algoritmos de implementação mais simples, é quadrático apenas no pior caso. [1]

Para manipular o dados foi criada uma classe Ponto que guarda as coordenadas x e y de cada ponto. A função que determina a envoltória convexa tem como entrada uma lista de pontos. A saída é uma lista dos pontos que formam a envoltória.

As funções implementadas e suas respectivas documentações estão no arquivo envoltoria.py.


In [70]:
from envoltoria import *

## Interseção de segmentos

Para verificar se há interseção de segmentos, utilizamos o algoritmo da varredura linear.

Para manipular os dados, foram criadas duas classes: Segmento e Evento. A classe Segmento guarda dois elementos da classe Ponto, o ponto inicial e o ponto final do segmento. A classe Evento representa um ponto de início ou de fim de um segmento, e guarda um elemento Ponto, uma string identificando o ponto como início ou fim de um segmento e um índice, que indica a qual segmento esse Evento pertence em uma lista de segmentos.

As funções implementadas e suas respectivas documentações estão no arquivo varredura.py.

In [ ]:
from varredura import*

## Separabilidade linear dos dados

** O que tá escrito na descrição do tp:
3. Implementar o método para verificação de separabilidade linear conforme descrito acima

Para verificar a separabilidade dos dados, o método deverá computar as envoltórias convexas de cada classe (será assumido que os dados possuem apenas dois rótulos para simplificar). Em seguida, deverá ser verificado se há ou não sobreposição dessas classes/envoltórias através do algoritmo de varredura linear para verificação de interseção de segmentos. Caso não haja interseção entre segmentos de envoltórias
distintas, então os dados são linearmente separáveis.
**


## Modelo e classificador

Primeiramente, encontramos a envoltória convexa dos pontos de cada classe. Em seguida, calcula-se a distância dos pontos entre as envoltórias para encontrar a distância mínima. O modelo do classificador é a reta perpendicular ao ponto médio do segmento que liga os dois pontos encontrados.

As funções para calcular o modelo e criar o classificador estão definidas e documentadas no arquivo modelo.py.

In [71]:
from modelo import *

O Iris é um dataset clássico para o estudo de ciência de dados, e por isso vamos utilizá-lo para testar nosso classificador. Como nosso algoritmo aceita apenas duas features e duas classes, precisamos filtrar o dataset.

Abaixo, fazemos com que o nosso dataset tenha apenas duas variáveis que descrevem as amostras: o comprimento e a largura da sépala de uma flor, respectivamente. Cada variável é uma coordenada no plano cartesiano.

As classes escolhidas são as variações Iris setossa e Iris versicolor. 

In [72]:
from sklearn.datasets import load_iris

iris = load_iris()
data_features = iris.data[:100, :2]
data_target = iris.target[:100]

print("Nomes das features:")
print(iris.feature_names)
print("Nomes das classes:")
print(iris.target_names)

Nomes das features:
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Nomes das classes:
['setosa' 'versicolor' 'virginica']


A seguir, convertemos as features para a classe Ponto.

In [73]:
data = lista_pontos(data_features)

Para medirmos o desempenho do classificador vamos utilizar a biblioteca Scikit-Learn. Ela fornecerá ferramentas para separar os dados em treino e teste e também as funções de cálculo das métricas de avaliação.

Primeiramente, dividiremos o dataset em 70% para treino e 30% para teste.

In [74]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(data, data_target, test_size=0.30)

Separamos os pontos das duas classes no conjunto de treino:

In [75]:
setosa = []
versicolor = []
for i in range(len(target_train)):
    if target_train[i] == 0:
        setosa.append(data_train[i])
    else:
        versicolor.append(data_train[i])

Calculamos a envoltória convexa de cada classe e calculamos o modelo. a e b são o coeficiente angular e linear da reta do modelo, respectivamente.

In [78]:
env_setosa = envoltoria_convexa(setosa)
env_versicolor = envoltoria_convexa(versicolor)
a, b = equacao_modelo(env_setosa, env_versicolor)

Abaixo, podemos ver que a espécie Iris setosa fica acima da reta do modelo, e portanto é da classe azul, e a Iris versicolor fica abaixo da reta, na classe vermelha. Isso significa que a classe azul corresponde à classe 0 no dataset, e a vermelha corresponde à classe 1.

In [81]:
print(classificador(setosa[0], a, b))
print(classificador(versicolor[0], a, b))

blue
red


Agora que temos o modelo, podemos alimentar os dados de teste e medir o quanto o classificador acerta.

In [85]:
classificacao = []
for i in range(len(target_test)):
    classe = classificador(data_test[i], a, b)
    if classe == 'blue':
        classificacao.append(0)
    else:
        classificacao.append(1)

Utilizaremos as métricas de precisão, revocação e score F1.

In [86]:
from sklearn.metrics import precision_score, recall_score, f1_score

precisao = precision_score(target_test, classificacao)
revocacao = recall_score(target_test, classificacao)
f1 = f1_score(target_test, classificacao)

print(precisao, revocacao, f1)

0.95 1.0 0.9743589743589743


Considerando que para as três métricas o pior valor é 0 e o melhor valor é 1, podemos dizer que o classificador funciona muito bem para esta base de dados.

Referências:

[1] O'Rourke, capítulo 3. Disponível em: https://www.cs.jhu.edu/~misha/Spring16/07.pdf